### ~~This work has been deprioritised due to concerns about scope creep~~
~~We may wish to revisit this work in a future CNV paper.~~

# This work has been reprioritised to generate the CRT breakpoints needed for faceaway reads analysis

### The code immediately below was deprioritised so it's a bit outdated and did not directly contribute to Pf8, but I later had to pick it back up again for Pf8's faceaway analysis (further below in section `Collecting breakpoint evidence for each sample as part of Pf8's faceaway analysis`)

In [ ]:
import re

import igv_notebook
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from IPython.display import Image
from plotly.subplots import make_subplots

def plot_cnv_diagnostic(sample, gene):
    display(Image(filename = f"<INSERT PATH HERE>/malariagen-pf8-cnv-calling/04_gcnv_calls_validation/results/{gene}/{sample}/plot.png"))

def igv(sample, locus):
    REF = "<INSERT PATH HERE>/resource-bundle/PlasmoDB-54_Pfalciparum3D7_Genome.fasta"
    GFF = "<INSERT PATH HERE>/resource-bundle/snpEff/data/Pfalciparum3D7_PlasmoDB_55/PlasmoDB-55_Pfalciparum3D7.gff"
    meta = pd.read_csv("../../assets_pf8/01_paths_to_bams.tsv", sep = "\t", names = ["SAMPLE", "PATH"])

    assert sample in meta.SAMPLE.tolist()

    path_to_bam = meta.loc[meta.SAMPLE == sample, "PATH"].values[0]
    
    igv_notebook.init()
    
    b = igv_notebook.Browser({
        "reference": {
            "name": "Plasmodium falciparum (3D7)",
            "fastaPath": REF,
            "indexed": False
        },
        "locus": locus
    })
    
    b.load_track({
        "name": "genes",
        "type": "annotation",
        "format": "gff3",
        "path": GFF,
        "displayMode": "COLLAPSED",
    })
    
    b.load_track({
        "path": path_to_bam,
        "indexPath": path_to_bam + ".bai",
        "format": "bam",
        "type": "alignment",
        "displayMode": "EXPANDED",
        "height": 500,
        "squishedRowHeight": 5,
        "viewAsPairs": "true",
        "showSoftClips": "true"
    })

meta = pd.read_csv("../../assets_pf8/Pf_8_samples_20241212.txt", sep = "\t", usecols = ["Sample", "Study", "Country", "Year", "Population"])
meta.Year = meta.Year.fillna(-1).astype(int)

pd.set_option("display.max.rows", None)
calls = pd.read_csv("../../04_gcnv_calls_validation/2024_12_02_pf8_coverage_calls.tsv", sep = "\t", usecols = ["SAMPLE", "CRT"]).rename(columns = {"SAMPLE": "Sample"})
calls = (
    calls.reset_index(drop = True)
    .merge(meta, on = "Sample")
    .merge(pd.read_csv("full_fws_20241125.tsv", sep = "\t"), on = "Sample")
    .merge(pd.read_csv("pf-haploatlas-PF3D7_0709000_sample_summary.csv", usecols = ["Sample", "HaploAtlas exclusion reason", "ns_changes"]))
)

# Show relevant metadata (from pf-haploatlas and pf8 sample metadata) for CRT duplications
calls.loc[calls.CRT == 1]

In [ ]:
print("Figure 1: Left breakpoint shows lots of soft clips and green reads (face-away reads, which suggests tandem duplication). Right breakpoint shows loads of green lining up nicely, within MED14. These are the large majority of samples, so if they haven't been analysed below, assume they have this breakpoint. ")
plot_cnv_diagnostic("PA0193-C", "CRT")
igv("PA0193-C", "Pf3D7_07_v3:398,565-399,609")
igv("PA0193-C", "Pf3D7_07_v3:415,952-426,679")

In [ ]:
print("Figure 1a: Same breakpoints as Figure 1, but is supposedly a triplication. No new evidence from IGV to explain mechanism of triplication though. Burkina Faso. gDNA. Het SNPs appear to be in 1:2 ratio")
plot_cnv_diagnostic("PK0019-C", "CRT")
igv("PK0019-C", "Pf3D7_07_v3:398,565-399,609")
igv("PK0019-C", "Pf3D7_07_v3:415,952-426,679")

In [ ]:
print("Figure 1b: Also a triplication. Ghana. gDNA. Het SNPs appear to be in 1:2 ratio")
plot_cnv_diagnostic("PF0392-C", "CRT")
igv("PF0392-C", "Pf3D7_07_v3:398,565-399,609")
igv("PF0392-C", "Pf3D7_07_v3:415,952-426,679")

In [ ]:
print("Figure 1c: Also a triplication. Mali. gDNA. Het SNPs appear to be in 1:2 ratio")
plot_cnv_diagnostic("PM0313-C", "CRT")
igv("PM0313-C", "Pf3D7_07_v3:398,565-399,609")
igv("PM0313-C", "Pf3D7_07_v3:415,952-426,679")

In [ ]:
print("Figure 1d: From Laos. This is a strange case though. Potentially DTD? Looks a bit different to Figure 1, but that may be because it's sWGA. The left breakpoint is a little messy to make out, but it helps to use look at the IGV plot for the right breakpoint, scroll down, and look at the mate start values for the faceaway reads there (Look for polyT red). I think it's safest here to assume sWGA is making it appear like a DTD, when really, it's the classic SCO1-MED14 breakpoint. Also, het SNPs are typically in a 50:50 or 55:45 ratio. ")
plot_cnv_diagnostic("RCN09132", "CRT")
igv("RCN09132", "Pf3D7_07_v3:394,081-405,493")
igv("RCN09132", "Pf3D7_07_v3:415,952-426,679")

In [ ]:
print("Figure 2: Different breakpoint to Figure 1. S8e-Cg7. The right breakpoint is further along than MED14, in the middle of Cg7. From Vietnam. ")
plot_cnv_diagnostic("RCN08130", "CRT")
igv("RCN08130", "Pf3D7_07_v3:364,093-368,572")
igv("RCN08130", "Pf3D7_07_v3:415,952-426,679")

In [ ]:
print("Figure 2a: Another S8e-Cg7. From Vietnam. Figures 2 and 2a are the only samples here that are homozygous duplications. All others have het duplications")
plot_cnv_diagnostic("RCN08145", "CRT")
igv("RCN08145", "Pf3D7_07_v3:364,093-368,572")
igv("RCN08145", "Pf3D7_07_v3:415,952-426,679")

In [ ]:
print("Figure 3: COX4-MED14. From Kenya")
plot_cnv_diagnostic("SPT42947", "CRT")
igv("SPT42947", "Pf3D7_07_v3:389,424-391,836")
igv("SPT42947", "Pf3D7_07_v3:415,952-426,679")

In [ ]:
print("Figure 4: HSP110-MED14. From Ghana")
plot_cnv_diagnostic("PF0219-C", "CRT")
igv("PF0219-C", "Pf3D7_07_v3:392,203-394,615")
igv("PF0219-C", "Pf3D7_07_v3:415,952-426,679")

---
# Making breakpoints file

In [ ]:
breakpoints_calls = {
    "SCO1-MED14-triplication": {
        "Coords": "Pf3D7_07_v3:398,900-421,500",
        "Samples": [
            "PK0019-C", "PF0392-C", "PM0313-C"
        ]
    },
    "COX4-MED14": {
        "Coords": "Pf3D7_07_v3:390,500-421,500",
        "Samples": [
            "SPT42947"
        ]
    },
    "HSP110-MED14": {
        "Coords": "Pf3D7_07_v3:390,500-421,500",
        "Samples": [
            "PF0219-C"
        ]
    },
    "S8e-Cg7": {
        "Coords": "Pf3D7_07_v3:366,000-424,800",
        "Samples": [
            "RCN08130", "RCN08145"
        ]
    },
    "SCO1-MED14": {
        "Coords": "Pf3D7_07_v3:398,900-421,500",
        "Samples": "*"
    },
}

In [ ]:
# By looking at various IGV plots, I built up this dictionary which stores information of all the different CRT breakpoints I could find. 
# Below, we use the dictionary to modify the sample metadata to provide breakpoint information, before doing some summary statistics. 

crt_breakpoints = calls.copy(deep = True)

crt_breakpoints.loc[crt_breakpoints.CRT != 1, "CRT_breakpoint"] = "-"

for k, v in breakpoints_calls.items():
    breakpoint_name   = k
    breakpoint_coords = v["Coords"]
    samples           = v["Samples"]

    if samples != "*":
        crt_breakpoints.loc[crt_breakpoints.Sample.isin(samples), "CRT_duplication_type"] = breakpoint_name
        crt_breakpoints.loc[crt_breakpoints.Sample.isin(samples), "CRT_breakpoint"]       = breakpoint_coords

    if samples == "*":
        crt_breakpoints.loc[crt_breakpoints.CRT == 1, "CRT_duplication_type"] = crt_breakpoints.loc[crt_breakpoints.CRT == 1, "CRT_duplication_type"].fillna(breakpoint_name)
        crt_breakpoints.loc[crt_breakpoints.CRT_duplication_type == breakpoint_name, "CRT_breakpoint"] = breakpoint_coords
        
        break # make sure this step gets performed last, otherwise fillna
    
crt_breakpoints.CRT = crt_breakpoints.CRT.map({0: "no_dup", 1: "dup", -1: "uncallable"})

pd.set_option("display.max_colwidth", None)

crt_breakpoints.loc[crt_breakpoints.CRT == "dup"].groupby("CRT_duplication_type").apply(lambda s: pd.Series({
    "Breakpoint coordinates": s.CRT_breakpoint.values[0],
    "Country": s.Country.value_counts().to_dict(),
    "Year": s.Year.value_counts().to_dict(),
    "Study": s.Study.value_counts().to_dict(),
}), include_groups=False).reset_index()

In [ ]:
def process_ns_changes(s):
    """For processing ns_changes from pf-haploatlas data"""
    l_l = [[int(num) for num in re.findall(r'\d+', l)] for l in s.split(",")]

    if len(l_l) == 1:
        if "*" in s:
            return {"het": l_l[0]}
        if "*" not in s:
            return {"hom": l_l[0]}

    if len(l_l) == 2:
        set1 = set(l_l[0])
        set2 = set(l_l[1])

        phase1_only = list(set1 - set2)
        phase2_only = list(set2 - set1)
        both_phases = list(set2 & set1)

        return {
            "het": phase1_only + phase2_only,
            "hom": both_phases
        }

dup_analysis = crt_breakpoints.loc[crt_breakpoints.CRT == "dup"].reset_index(drop = True).copy(deep = True)
dup_analysis["int_aa_changes"] = dup_analysis.ns_changes.apply(process_ns_changes)

for i, row in dup_analysis.iterrows():
    for colour, l_int in row.int_aa_changes.items():
        dup_analysis.loc[dup_analysis.Sample == row.Sample, l_int] = colour

aa_columns    = sorted([col for col in dup_analysis.columns if isinstance(col, int)])
other_columns = [col for col in dup_analysis.columns if isinstance(col, str)]

dup_analysis = dup_analysis[other_columns + aa_columns].sort_values(["CRT_duplication_type", "Population", "Country", 74, 326, 371])

In [ ]:
# Please be aware that while preparing this codebase, we were also battling with some bugs in `call_haplotypes` and pf-haploatlas, so the data is not quite right here yet. We are missing Q271E.

fig = make_subplots(
    rows=1, cols=5, 
    subplot_titles=["Population", "Country", "Clonality", "CRT breakpoint", "CRT haplotype SNP position"],
    shared_yaxes=True,
    column_widths=[1, 1, 1, 2, 8],
    horizontal_spacing=0.005
)

population_colours = {
    "SA": "#4daf4a",
    "AF-W": "#e31a1c",
    "AF-C": "#fd8d3c",
    "AF-NE": "#bb8129",
    "AF-E": "#fecc5c",
    "AS-S-E": "#dfc0eb",
    "AS-S-FE": "#984ea3",
    "AS-SE-W": "#9ecae1",
    "AS-SE-E": "#3182bd",
    "OC-NG": "#f781bf",
}

population_index_map = {pop: i for i, pop in enumerate(dup_analysis.Population.unique())}
population_colourscale = [[i / (dup_analysis.Population.nunique()-1), population_colours[pop]] for i, pop in enumerate(dup_analysis.Population.unique())]

fig.add_trace(
    go.Heatmap(
        z=dup_analysis.Population.map(population_index_map),
        x=[""] * len(dup_analysis),
        y=dup_analysis.Sample.values,
        colorscale = population_colourscale,
        showscale=False,
        text = dup_analysis.Population,
        texttemplate="%{text}",
        hoverinfo="none",
        ygap=1
    ),
    row=1, col=1
)

country_index_map = {country: i for i, country in enumerate(dup_analysis.Country.unique())}
country_colourscale = [[i / (dup_analysis.Country.nunique()-1), px.colors.qualitative.Bold[i]] for i, country in enumerate(dup_analysis.Country.unique())]

fig.add_trace(
    go.Heatmap(
        z=dup_analysis.Country.map(country_index_map),
        x=[""] * len(dup_analysis),
        y=dup_analysis.Sample.values,
        colorscale = country_colourscale,
        showscale=False,
        text = dup_analysis.Country,
        texttemplate="%{text}",
        hoverinfo="none",
        ygap=1
    ),
    row=1, col=2
)



fws_colourscale = [
    [1, "rgba(59, 224, 88, 0.8)"], [0, "rgba(216, 238, 220, 0.8)"]
]

fig.add_trace(
    go.Heatmap(
        z=pd.cut(dup_analysis.fws, [0, 0.95, 1], labels = [0, 1]).astype(int),
        x=[""] * len(dup_analysis),
        y=dup_analysis.Sample.values,
        colorscale = "ice",
        showscale=False,
        text = pd.cut(dup_analysis.fws, [0, 0.95, 1], labels = ["Clonal", "Mixed"]),
        texttemplate="%{text}",
        hoverinfo="none",
        ygap=1
    ),
    row=1, col=3
)


dup_type_index_map = {dup_type: i for i, dup_type in enumerate(dup_analysis.CRT_duplication_type.unique())}
dup_type_colourscale = [[i / (dup_analysis.CRT_duplication_type.nunique()-1), px.colors.qualitative.Pastel[i]] for i, dup_type in enumerate(dup_analysis.CRT_duplication_type.unique())]

fig.add_trace(
    go.Heatmap(
        z=dup_analysis.CRT_duplication_type.map(dup_type_index_map),
        x=[""] * len(dup_analysis),
        y=dup_analysis.Sample.values,
        colorscale = dup_type_colourscale,
        showscale=False,
        text = dup_analysis.CRT_duplication_type,
        texttemplate="%{text}",
        hoverinfo="none",
        ygap=1
    ),
    row=1, col=4
)


haplotype_colourscale = [
    [0, "#99ccff"],
    [1, "#ffcccc"]
]

fig.add_trace(
    go.Heatmap(
        z=dup_analysis[aa_columns].map(lambda x: {np.nan: np.nan, "het": 1, "hom": 2}[x]),
        x=[str(col) for col in aa_columns],
        y=dup_analysis.Sample.values,
        colorscale=haplotype_colourscale,
        showscale=False,
        text=dup_analysis[aa_columns].map(lambda x: {np.nan: "", "het": "Het", "hom": "Hom"}[x]).values,
        texttemplate="%{text}",
        hoverinfo="none",
        ygap=1
    ),
    row=1, col=5
)
fig.update_xaxes(ticklen=10, tickcolor="blue", tickwidth=2)
fig.update_xaxes(showgrid=False, row=1, col=4)

fig.update_layout(
    height=1000,
    width=1500,
    template="plotly_white"
)

fig.show()

---
---
---

# Collecting breakpoint evidence for each sample as part of Pf8's faceaway analysis

#### The code below here is very scrappy, but I used it to hunt for tandem duplication breakpoints found in `tandem_dup_breakpoints_20250128.txt`

In [ ]:
final_breakpoints_calls = {
    "SCO1-MED14": {
        "left_breakpoint" : "Pf3D7_07_v3:398,869-398,898",
        "right_breakpoint": "Pf3D7_07_v3:421,765-421,808",
        "Samples": [
            "PA0193-C", "PF0054-C", "PF0118-C", "PF0190-C", "PF0195-C", "PF0231-C", "PF0250-C",
            "PF0356-C", "PF0449-C", "PF0590-C", "PF0729-C", "PF0796-C", "PF0852-C", "PF0905-C",
            "PF0934-C", "PF0960-C", "PF0961-C", "PF0987-C", "PF1103-C", "PK0023-C", "SPT22565",
            "PK0063-C", "PK0066-C", "PK0068-C", "PK0075-C", "PM0124-C", "PM0243-C", "PM0336-C",
            "PM0345-C", "PM0455-C", "PM0476-C", "PM0496-C", "PM0580-C", "QQ0097-C", "QV0096-C",
            "SPT00802", "SPT26595", "SPT26638", 
        ]
    },
    "SCO1-MED14-Laos": {
        "left_breakpoint" : "Pf3D7_07_v3:399,263-399,291",
        "right_breakpoint": "Pf3D7_07_v3:422,103-422,136",
        "Samples": [
            "RCN09132", "RCN11806", "RCN15231", "RCN15335"
        ]
    },
    "COX4-MED14": {
        "left_breakpoint" : "Pf3D7_07_v3:390,397-390,416",
        "right_breakpoint": "Pf3D7_07_v3:421,765-421,808",
        "Samples": [
            "SPT42947"
        ]
    },
    "HSP110-MED14": {
        "left_breakpoint" : "Pf3D7_07_v3:393,103-393,133",
        "right_breakpoint": "Pf3D7_07_v3:421,765-421,808",
        "Samples": [
            "PF0219-C"
        ]
    },
    "S8e-Cg7": {
        "left_breakpoint" : "Pf3D7_07_v3:365,968-365,991",
        "right_breakpoint": "Pf3D7_07_v3:425,216-425,236",
        "Samples": [
            "RCN08130", "RCN08145"
        ]
    },
    "SCO1-MED14-triplication": {
        "left_breakpoint" : "Pf3D7_07_v3:398,869-398,898",
        "right_breakpoint": "Pf3D7_07_v3:421,765-421,808",
        "Samples": [
            "PF0392-C", "PM0313-C", "PK0019-C"
        ]
    },
    "no_precise_break_point": {
        "left_breakpoint" : "",
        "right_breakpoint": "",
        "Samples": [
            "PF0149-C", "PK0069-C"
        ]
    },
}

In [ ]:
final_breakpoints = crt_breakpoints.loc[crt_breakpoints.CRT == "dup", ["Sample", "Country", "Year", "Population", "fws", "CRT_breakpoint", "CRT_duplication_type"]].reset_index(drop = True)

for k, v in final_breakpoints_calls.items():
    final_breakpoints.loc[final_breakpoints.Sample.isin(v["Samples"]), "left_breakpoint"]  = v["left_breakpoint"]
    final_breakpoints.loc[final_breakpoints.Sample.isin(v["Samples"]), "right_breakpoint"] = v["right_breakpoint"]

top_sample = final_breakpoints.loc[pd.isnull(final_breakpoints[["left_breakpoint", "right_breakpoint"]]).any(axis = 1), "Sample"].values[0]

final_breakpoints.loc[pd.isnull(final_breakpoints.left_breakpoint)].head()

In [ ]:
print(top_sample)

plot_cnv_diagnostic(top_sample, "CRT")
igv(top_sample, "Pf3D7_07_v3:398,724-399,043")
igv(top_sample, "Pf3D7_07_v3:421,611-421,962")

# igv(top_sample, "Pf3D7_07_v3:399,263-399,291")
# igv(top_sample, "Pf3D7_07_v3:422,103-422,136")

In [ ]:
final_breakpoints

In [ ]:
igv("PD1033-C", "Pf3D7_05_v3:928,340-928,359")

In [ ]:
igv("PJ0266-C", "Pf3D7_12_v3:939,000-940,000")